# Tile staging

In [ ]:
import sys
sys.path.append('..')

from multiview_stitcher import spatial_image_utils as si_utils
import napari

from muvis_align.MVSRegistration import MVSRegistration
from muvis_align.util import dir_regex
from muvis_align.image.util import get_sim_physical_size, get_sim_shape_2d

## Setup parameters

In [ ]:
params = {
	'general':
	{
        'ui': 'mpl',
        'debug': True
	},
	'operations':
	[
		{
            'operation': 'register match s',
            'input': 'D:/slides/EM04652-02_slice17_spaghettiandmeatballs2/**/*_s00400.ome.tif',
            'source_metadata': 'sbemimage',
            'use_orthogonal_pairs': True,
            'fusion': 'exclusive',
    		'output_spacing': 'max',
            'n_parallel_pairwise_regs': 1,
			'output': '../../output/'
		}
	]
}


## Initialise muvis-align, initialise sims

In [ ]:
operation_params = params['operations'][0]
filenames = dir_regex(operation_params['input'])

reg = MVSRegistration(params['general'])
reg.init_operation('', filenames, operation_params)
sims = reg.init_sims()

for label, sim in zip(reg.file_labels, sims):
    print(label, si_utils.get_origin_from_sim(sim), get_sim_physical_size(sim))

## Calculate tile shapes

In [ ]:
shapes = [get_sim_shape_2d(sim, transform_key=reg.source_transform_key) for sim in sims]
for label, shape in zip(reg.file_labels, shapes):
	print(label, shape[0])

## Calculate fused image

In [ ]:
fused, _ = reg.fuse(sims, transform_key=reg.source_transform_key)
fused_scale = si_utils.get_spacing_from_sim(fused, asarray=True)
fused_position = si_utils.get_origin_from_sim(fused, asarray=True)

## Open napari viewer

In [ ]:
viewer = napari.Viewer()

## Visualise tiles and fused image

In [ ]:
viewer.add_image(fused, name='fused', scale=fused_scale, translate=fused_position)
text = {'string': '{labels}', 'color': 'blue'}
features = {'labels': reg.file_labels}
viewer.add_shapes(shapes, name=f'tile shapes', text=text, features=features, face_color='transparent')